In [1]:
#archivo para crear las tablas de inputs requeridas para que funcione el algoitmo de optimización por producto 

using DataFrames, CSV

#llamamos a las bases de datos que vamos a utilizar
restricciones = CSV.read("restricciones de basicos.csv")
res = convert(Array, restricciones)
grupos = CSV.read("basicos por grupo.csv")
grup = convert(Array{Any,2}, grupos)
basicos = CSV.read("info basicos.csv")
bas = convert(Array, basicos)
precios = CSV.read("resumen de precios.csv")
pre = convert(Array, precios)
formulaciones = CSV.read("formulaciones.csv")
f = convert(Array{Any,2},formulaciones)

#identificamos el producto a tratar 

print("indique el código de producto que desea analizar")
z = parse(Int32,readline(STDIN))
z_1 = string(z)


y = findfirst(f[:,3],z)


product = formulaciones[y,:]
println(product)

#encontramos las restriccciones de básicos de este producto 
y = findfirst(res[:,1],z_1)


y1 = res[y,6:10]

lg = length(grup[:,1]) #nos indica cual es la cantidad máxima de básicos por grupo

g = Array{Any,2}(lg,5) #se indica 5 porque es la cantidad de grupos posibles en el caso de haber mas o menos este numero debe cambiar

#obtenemos los básicos especificados en la formulación 
y2 = res[y,2:5]
fun(y2) = y2 == 0
zero_elements = find(fun,y2)
y3 = deleteat!(y2,zero_elements)

#obtenemos todos los básicos permitidos para hacer este producto 
nss = 0 
for i = 1:5
    nss = nss+1
    if y1[i]==1 
        g[:,nss]=grup[:,2*i]
        else 
        g[:,nss]= zeros(Int8,lg,1)
        end 
    end 


    
#find the zero elements in the arrays and delete them 
basico = Array{Any,1}(5)
ns = 0
for i = 1:5
    ns = ns+1
    x = g[:,i]
    fun(x) = x == 0
    zero_elements = find(fun,x)
    new_vector = deleteat!(g[:,i],zero_elements)
    basico[ns] = new_vector
end

#construimos un vector con todos los básicos permitidos para formar este producto
b1 = append!(basico[1],basico[2])
b2 = append!(b1,basico[3])
b3 = append!(b2,basico[4])
b4 = append!(b3,basico[5])

na = 0
fin = Array{Any,1}(length(y3))
#agregamos a este vector los básicos especificados por formula, si ya se encuentran en este vector, no se agregan
for k = 1:length(y3)
    na = na+1
    fin = findfirst(b4,y3[k])
    
    if fin > 0 
        else fin == 0 
        push!(b4,y3[k])
    end
        
    end 
   
#ahora hay que encontrar la información de costo, viscosidad dinámica a 100°C, viscosidad cinemática @T de producto, cantidad máxima que se puede comprar, 

#obtenemos los datos de la hoja de formulaciones [temp de ccs, color max, volatilidad max ]
indice = findfirst(f[:,3],z)
temp_ccs = f[indice,11]
colorf = f[indice,13]
volatf = f[indice, 14]
viscdintar= f[indice,12]
visckintar = f[indice,10]


#creamos los vectores donde se guardará la información 
cost = Array{Float64,1}(length(b4))
visckin = Array{Float64,1}(length(b4))
ccs = Array{Float64,1}(length(b4))
color = Array{Float64,1}(length(b4))
volat = Array{Float64,1}(length(b4))
#las banderas que indican si hay o no restricciones de color, volatilidad y visckin
flagccs = Array{Int8,1}(1)
flagcolor = Array{Int8,1}(1)
flagvolat = Array{Int8,1}(1)

#anclamos los contadores
nss = 0 
ns = 0 
na = 0 

for i = 1:length(b4)
    nss = nss+1
    #indice de costo en "resumen precios.csv" 
    indc=findfirst(pre[:,2],b4[i])
    #costo de los basicos 
    cost[nss]= precios[indc,5]
    #indice de básico en "info basicos.csv"
    indbas = findfirst(bas[:,2],b4[i])
    #viscosidad dinamica a 100°C de los basicos 
    visckin[nss] = bas[indbas,3]
    
    
    #hay cierta información que es requerida o no de acuerdo a las especificaciones del producto. En este caso, la temperatura a la cual se mide el ccs indica q datos iniciales requerimos 
    if temp_ccs == 0
        flagccs = 0 #no todoslos productos toman en cuenta las propiedades a baja temp. en este caso dejamos el vector en 0 
    elseif temp_ccs == -10 
        ccs[nss] = bas[indbas,9]
        flagccs = 1
    elseif temp_ccs == -15
        ccs[nss] = bas[indbas,10]
        flagccs = 1
    elseif temp_ccs == -20
        ccs[nss] = bas[indbas, 11]
        flagccs = 1
    elseif temp_ccs == -25 
        ccs[nss] = bas[indbas, 12]
        flagccs = 1
    elseif temp_ccs == -30 
        ccs[nss] = bas[indbas, 13]
        flagccs = 1
    elseif temp_ccs == -35
        ccs[nss] = bas[indbas,14]
        flagccs = 1
        end 
    #también, para ciertos productos se requiere especificación de color y volatilidad 
    if colorf == 0 
        flagcolor = 0 
        else 
        ns = ns + 1 
        color[ns] = bas[indbas,6]
        flagcolor = 1
        end 
    if volatf == 0 
        flagvolat = 0 
        else 
        na = na + 1
        volat[na]= bas[indbas,7]
        flagvolat = 1
        end 
end

flags = [flagccs flagcolor flagvolat]
#salvamos los flags

#ahora extraemos los datos del input del usuario 


info = DataFrame(basico = b4, Visc_dinamica = ccs, visc_cinematica_100C = visckin, volatilidad = volat, col = color, costo_mxn = cost) 


#vectores donde esta guardada la info 
#b4 --- codigos de basicos permitidos 
#cost --- costo de basicos permitidos 
#visckin --- viscosidad kinematica a 100°C de basicos permitidos 
#ccs --- viscosidad dinamica a temp de producto de basicos permitidos (en el caso de ser requerido por el producto )
#color --- color de basicos permitidos (en el caso de ser requerido por el producto)
#volat --- volatilidad de basicos permitidos (en el caso de ser requerido por el producto)
    

indique el código de producto que desea analizarSTDIN> 2124
1×15 DataFrames.DataFrame. Omitted printing of 10 columns
│ Row │ linea │ grado │ codigo producto │ B1 nombre               │ B1 codigo │
├─────┼───────┼───────┼─────────────────┼─────────────────────────┼───────────┤
│ 1   │ GAS   │ MONO  │ 2124            │  SOLVENT NEUTRAL 600 GI │ 3006      │


,basico,Visc_dinamica,visc_cinematica_100C,volatilidad,col,costo_mxn
1,3021,5.69953e-316,4.45,5.69953e-316,1.5,11.81
2,3007,5.69953e-316,6.73,5.69953e-316,3.0,67.47
3,3003,5.69953e-316,16.15,5.69953e-316,6.0,49.48
4,3603,2.30977e-315,7.12,5.69953e-316,1.0,71.47
5,3006,2.30977e-315,12.17,5.69953e-316,2.0,15.76
6,3017,5.69953e-316,31.99,5.69953e-316,4.0,13.14
7,3601,5.69953e-316,37.97,5.69953e-316,3.5,11.54
8,3044,5.69953e-316,4.67,5.69953e-316,0.5,12.74
9,3043,5.69953e-316,6.57,5.69953e-316,0.5,59.61
10,3048,5.69953e-316,11.93,5.69953e-316,0.5,16.4


In [2]:
product

,linea,grado,codigo producto,B1 nombre,B1 codigo,B1 X1,B2 nombre,B2 codigo,B2 X2,cSt a 100C,temp para CCS,cP sin mejorador,Color,Volatilidad,cP con mejorador
1,GAS,MONO,2124,SOLVENT NEUTRAL 600 GI,3006,0.3637,SOLVENT NEUTRAL 250,3603,0.6363,8.6,0,0,4.5,0,0
